In [76]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

# 벡터 DB : Chroma vs Pinecone

- Chroma : 인메모리DB, 로컬메모리 DB
- Pinecone : 클라우드 vector DB
  - [Pinecone console 에 api key 생성 -> .env (PINECONE_API_KEY)]
- pip install langchain-pinecone
- pip install pinecone-client pinecone-text


In [77]:
# %pip install langchain-pinecone pinecone-client pinecone-text

# 1. Knowledge Base 구성을 위한 데이터 생성


In [78]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [79]:
len(document_list)

183

In [80]:
# embedding : upstage embedding-query
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large",
    # model="embedding-query"
)

In [81]:
# pinecon vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()

# 데어터를 처음 업로드할 떄
index_name = "tax-index-upstage"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embeddings,
#     index_name=index_name,
# )

In [82]:
# 업로드한 벡터DB 가져올 떄
database = PineconeVectorStore(
    embedding=embeddings, # 질문에 입배딩하여 유사도 검색
    index_name=index_name,
)


# 2. 답변 생성을 위한 Retrieval


In [83]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

retriever = database.as_retriever(
    # search_kwargs={"k": 10},
)

retriever.invoke(query)


[Document(id='89e4f9c1-24dd-4a2a-8c92-7742f4db7bbd', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는

# 3. 제동되는 prompt 를 활용하여 답변 생성


In [84]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-nano")

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever, # database.as_retriver()
)

In [85]:
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 공식적으로 계산된 세액이 아닌, 연 55만원 또는 55만원과 40만원의 합인 95만원 수준일 가능성이 높습니다. 그러나 정확한 세액은 각종 공제와 세율에 따라 달라지므로 구체적 계산이 필요합니다. 일반적으로 이 수준은 공제 후 세율 적용 전에 예상할 수 있는 대략적인 금액입니다.'}